In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Introduction
Vaccines today play an important role in sustaining community health, hence it is important to understant what kind of myths are circulated about them and where it originates. Categorising such myths into different topics can help us understand the origin of myth (eg mistrust of vaccine,religious stigma, psedosciece etc).Hence it is important for people to understand scientific and medical community to understand the core topics involvng such myhts to countert them effectively, and convince victims of such myths otherwise.

Topic modeling is a type of statistical modeling for discovering the abstract “topics” that occur in a collection of documents. Latent Dirichlet Allocation (LDA) is an example of topic model and is used to classify text in a document to a particular topic.

Here we'll try to perform EDA on a the reddit-vaccine-myth dataset and further try and classify them in topics.

![](https://www.hopkinsmedicine.org/-/media/images/health/1_-conditions/coronavirus/vaccine-hero.ashx?h=500&la=en&mh=500&mw=1300&w=1297&hash=ED466B472CD015FF5CC3288E75EA37EB707B65D3)

In [ ]:
data = pd.read_csv('../input/reddit-vaccine-myths/reddit_vm.csv', error_bad_lines=False);
data.head()


From the data its clear thast many posts have thier body missing, lets try to find out which all data are missing.

In [ ]:
data.isnull().sum(axis=0)

In [ ]:
data.shape

In [ ]:
sns.heatmap(data.isnull(),cbar= False)

From heatmap we can understand that url is missing in 69% of rows.

Since title is available in all rowns we'll use title to for preliminary topic modeling

#### Data Preprocessing
1. Tokenizing : the sentences are splint into words.
2. Stop word are removed
3. Lemmatization : past and future tenses are changed to present 
4. Stemming : words are reduced into their root form


#### Impoting necessary libaries for topic modeling

In [ ]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
from nltk import word_tokenize, pos_tag

nltk.download('wordnet')
stemmer = PorterStemmer()

In [ ]:
from nltk.corpus import wordnet

def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [ ]:
tokens = word_tokenize("hi this is ren")

nltk.pos_tag(tokens)


In [ ]:
def lemmatize_stemming(text):
   
    token=word_tokenize(text)
  
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, get_wordnet_pos(nltk.pos_tag(token)[0][1])))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [ ]:
preProcessedText = data["title"].apply(preprocess)
titlesWithoutDupes = data["title"].drop_duplicates()
titlesWithoutDupes

In [ ]:
preProcessedText
preProcessedTextWithoutComments = titlesWithoutDupes.apply(preprocess)
preProcessedTextWithoutComments 

In [ ]:
dictionary = gensim.corpora.Dictionary(preProcessedText)
dictionaryWithoutComents = gensim.corpora.Dictionary(preProcessedTextWithoutComments)

## LDA 
We'll apply the  Latent Dirichlet Allocation (LDA) to our documents and classify them into topics. Latent Dirichlet Allocation (LDA) is an example of topic model and is used to classify text in a document to a particular topic. It builds a topic per document model and words per topic model, modeled as Dirichlet distributions.

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in preProcessedText]
bow_corpusWithoutComments = [dictionaryWithoutComents.doc2bow(doc) for doc in preProcessedTextWithoutComments]

In [ ]:
lda_model = gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 4, 
                                   id2word = dictionary,                                    
                                   passes = 20,
                                   workers = 2)
lda_model.show_topics()

In [ ]:
lda_model_after_removing_dupes = gensim.models.LdaMulticore(bow_corpusWithoutComments,num_topics = 4,id2word = dictionaryWithoutComents,
                                                           passes = 20,workers=2)
lda_model_after_removing_dupes.show_topics()

### LDA visualization before removing duplicates from titltes

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models

LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, bow_corpus, dictionary)
pyLDAvis.display(LDAvis_prepared)

### LDA visualization after removing duplicates from titltes

In [ ]:
LDAvis_duples = pyLDAvis.gensim_models.prepare(lda_model_after_removing_dupes,bow_corpusWithoutComments,dictionaryWithoutComents)
pyLDAvis.display(LDAvis_duples)

it is evident that after removing duplicate titles from the corpus the model has perofrmed susbstantially better , the visualisation unlike the one with duplicates hs defined large, non overlaping circles , indicating that topics are different from each other and unique.

## Activity in subreddit
Lets take a look at the activity changes in the subredditt

In [ ]:
data["date"]= data["timestamp"].apply(lambda x: pd.Timestamp(x).strftime('%Y-%m-%d'))

In [ ]:
data

In [ ]:
activity = data["date"].value_counts()

In [ ]:
activity = pd.DataFrame(activity)
activity.reset_index(inplace=True)
activity["dates"] =activity["index"]
activity["count"] = activity["date"]
activity.drop(['index','date'],inplace=True,axis=1)

In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize=(15,8))

ax=sns.countplot(data=data, x='date')


In [ ]:
activity.sort_values(by="dates",inplace=True)

In [ ]:
plt.figure(figsize=(15,8))

lx= sns.lineplot(data= activity,x="dates",y="count")

In [ ]:
activity["year"] = activity['dates'].apply(lambda x:x[0:4])

In [ ]:
activity

In [ ]:
yearlyActivity = activity["year"].value_counts()


In [ ]:
yearlyActivity = pd.DataFrame(yearlyActivity)

In [ ]:
yearlyActivity.reset_index(inplace=True)
yearlyActivity["years"] = yearlyActivity["index"]
yearlyActivity["count"] = yearlyActivity["year"]

In [ ]:
plt.figure(figsize=(15,8))
yearlyActivity.sort_values(by="years",inplace=True)
lx= sns.lineplot(data= yearlyActivity,x="years",y="count")

From the graph it is evident that the activity in subreddit had peaked in 2019 and reduced significantly after the onset of covid-19 in 2020 and further decreasing in 2021

## LSA
LSA for natural language processing task was introduced by Jerome Bellegarda in 2005. The objective of LSA is reducing dimension for classification. The idea is that words will occurs in similar pieces of text if they have similar meaning. Now we'll try to use this also for classifying the titles.

In [ ]:

def detokenize(preProcessedTexts):
    detokenized_doc = []
    for i in range(len(preProcessedTexts)):
        t = ' '.join(preProcessedTexts[i])
        detokenized_doc.append(t)
    return(detokenized_doc)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

tfidf_vec = TfidfVectorizer(use_idf=True, norm='l2')
svd = TruncatedSVD(n_components=4)
detokenized_doc = detokenize(preProcessedText)
X= tfidf_vec.fit_transform((detokenized_doc))

In [ ]:
preProcessedText

In [ ]:
preProcessedTextWithoutComments

In [ ]:
detokenized_doc_no_comments = detokenize(list(preProcessedTextWithoutComments))
XWithoutComments= tfidf_vec.fit_transform((detokenized_doc_no_comments))

In [ ]:
X.shape


In [ ]:
x_svd = svd.fit_transform(X)
x_svd_noComment = svd.fit_transform(XWithoutComments)

In [ ]:
svd.fit(X)

In [ ]:
svd.components_

In [ ]:
import umap
embedding = umap.UMAP(n_neighbors=150, min_dist=0.5, random_state=12).fit_transform(x_svd)

plt.figure(figsize=(7,5))
plt.scatter(embedding[:, 0], embedding[:, 1], 

s = 10, # size
edgecolor='none'
)
plt.show()

In [ ]:
embedding = umap.UMAP(n_neighbors=150, min_dist=0.5, random_state=12).fit_transform(x_svd_noComment)

plt.figure(figsize=(7,5))
plt.scatter(embedding[:, 0], embedding[:, 1], 

s = 10, # size
edgecolor='none'
)
plt.show()

From the visualizations developed using both the methods, we can classify the documents into 4 topics.

1. vaccine causing autism and cancer
2. vaccine myth debunked and anti-vaxxer's childer need courts order for vaccine
3. big pharma paying scientist to promote vaccine
4. emergency occuring due to anti vaxx

## WordCloud

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


strings_no_comment = " ".join(detokenized_doc_no_comments)
wordcloud = WordCloud().generate(strings_no_comment)
plt.figure(figsize=(20,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")

plt.show()

From the wordcloud it cn be we get a clear idea of the words used by people in the subreddit. The frequency of the word "autism" shows the prevalnce of the idea of vaccines causing atism in the subreddit.

this notebook is still a work in progress , please give your valuable feedback so that I can improve it, do consider upvoting if you found it helpful.

credits:

https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

https://www.kaggle.com/abhishekvermasg1/analysis-topic-modelling